In [8]:
import json
import numpy as np
import os
os.chdir('/workspace/noise_and_dice/')

In [9]:
file_path = 'labels-heart.json'
with open(file_path, 'r') as file:
    labels = json.load(file)  # Reads and parses the JSON content

In [10]:
label_list = []
for key,val in labels.items():
    print(key, ': ', val)
    label_list.append(val)

label_list = np.sort(label_list)

heart :  1


In [11]:
print(label_list)

[1]


## Mapping TS labels to vista

In [12]:
file_path = 'heart-TS-map.json'  # specify the output file path

if os.path.exists(file_path):
    with open(file_path,'r') as json_file:
        label_TCIA_map = json.load(json_file)
else: 
    label_TCIA_map = {}
    for key,val in labels.items():
        label_TCIA_map[key] =  input(f'Enter TCIA segmentation name for: {key}') + '.nii.gz'

    with open(file_path, 'w') as json_file:
        json.dump(label_TCIA_map, json_file, indent=4)  # indent=4 for pretty-printing

In [13]:
import SimpleITK as sitk
import numpy as np
import tqdm
def calculate_mask_volume(mask,pixelvolume,label=1):
    counts = np.count_nonzero(mask == label)
    return counts*pixelvolume

In [14]:
scan_dir = '/processed/Public/UNOS/label_set_final'
label_dir = '/processed/Public/UNOS/label_set_final/predictions/TS'
out_dir = '/processed/Public/UNOS/label_set_final/predictions/TS_heart'

## Create labels for vista 3d comparison

In [15]:
scans = set([f for f in os.listdir(scan_dir) if f.endswith('.nii.gz')])
masks = set(([f for f in os.listdir(out_dir) if f.endswith('.nii.gz')]))

missing_labels = scans - masks
print('no of labels: ', len(masks))
print('no of scans: ', len(scans))
#print(missing_labels)


no of labels:  0
no of scans:  26


## Create labels for vista3d evaluation

In [16]:
exam_ids = [f.split('.')[0]  for f in os.listdir(scan_dir) if f.endswith('.nii.gz')]

dict_list = []

for eid in tqdm.tqdm(exam_ids):
    mask_out = os.path.join(out_dir,eid+'.nii.gz')
    dict_sequence = {}
    scan_file = os.path.join(scan_dir,eid+'.nii.gz')
    img = sitk.ReadImage(scan_file) 
    size = img.GetSize()
    rows = size[0]
    cols = size[1]
    slices = size[2]
    spacing = img.GetSpacing()
    pixell = spacing[0]
    pixelw = spacing[1]
    thickness = spacing[2]
    pixel_area = pixell*pixelw
    pixel_volume = pixel_area*thickness
    img_arr = sitk.GetArrayViewFromImage(img)
    pixel_low = np.min(img_arr)
    pixel_high = np.max(img_arr)
    pixel_mean = np.mean(img_arr)
    pixel_median = np.median(img_arr)

    #print(eid)
    dict_sequence['patid'] = eid
    dict_sequence['rows']= rows
    dict_sequence['cols']= cols
    dict_sequence['slices'] = slices
    dict_sequence['pixel_volume'] =pixel_volume
    dict_sequence['pixel_area'] =pixel_area
    dict_sequence['slice_thickness'] =thickness
    dict_sequence['intensity_low'] = pixel_low
    dict_sequence['intensity_high'] = pixel_high
    dict_sequence['intensity_mean'] = pixel_mean
    dict_sequence['intensity_median'] = pixel_median
    
    output_arr = None
    folder_path = os.path.join(label_dir,eid)
    for key,val in label_TCIA_map.items():
        mask_file = os.path.join(folder_path,val)
        segment_name = val.split('.')[0]
        vol_key = segment_name + '_volume'
        if os.path.exists(mask_file):
            mask = sitk.ReadImage(mask_file)
            mask_arr = sitk.GetArrayFromImage(mask) 
            mask_arr[mask_arr > 0] = 1
            dict_sequence[vol_key]= np.round(calculate_mask_volume(mask_arr,pixel_volume,label=1)/1000) # in ml
            mask_arr = mask_arr * labels[key]
            #print('vol_key: ', vol_key, ' volume: ', dict_sequence[vol_key])
            if output_arr is None:
                output_arr = mask_arr
            else:
                # Use pixel-wise maximum to merge without summing labels
                output_arr = np.maximum(output_arr, mask_arr)
        else:
            dict_sequence[vol_key] = 0
    
    # print('Liver and spleen volume calculated')
    if output_arr is not None:
        mask_sitk = sitk.GetImageFromArray(output_arr)
        mask_sitk.CopyInformation(img)
        sitk.WriteImage(mask_sitk, mask_out)
    
    dict_list.append(dict_sequence)

100% 26/26 [00:29<00:00,  1.12s/it]


In [ ]:
dict_list[0].keys()

In [ ]:
import pandas as pd
df_summary = pd.DataFrame(dict_list)

In [ ]:
list(df_summary)

In [ ]:
#display(df_summary)
#df_summary.to_csv(os.path.join('TCIA_ped_imagestats.csv'),index=False)